In [8]:
import jsonschema
import os 
import json

In [14]:
def calcualte_number_errors(path):
    with open(f'{path}/schema.json','r',encoding='utf-8') as f : 
        schema = json.load(f)
    witness_path = f'{path}/instance.json'
    if not os.path.exists(witness_path):
        return None 
    try : 
        with open(witness_path,'r',encoding='utf-8') as f : 
            data = json.load(f)
    except :
        return None
    try : 
        validator = jsonschema.Draft7Validator(schema)
        print(list(validator.iter_errors(data)))
        for error in validator.iter_errors(data): 
            print(error)
        return len(list(validator.iter_errors(data)))
    except Exception as e : 
        if("error: nothing to repeat at position 1" in str(e)) : # error appearing somehow
            return 0
        else :
            return None 
        
    
# calcualte_number_errors('o33194',f'./dataset/JE/{witness_file_name("o33194",True)}')

In [15]:
calcualte_number_errors('./dataset/2')

[<ValidationError: "{'a': {'b': 12}} is not valid under any of the given schemas">]
{'a': {'b': 12}} is not valid under any of the given schemas

Failed validating 'anyOf' in schema['properties']['name']:
    {'anyOf': [{'description': 'Cool! We got a string here!',
                'properties': {'a': {'minProperties': 2,
                                     'properties': {'b': {'type': 'string'}},
                                     'required': ['b'],
                                     'type': 'object'}},
                'required': ['a'],
                'type': 'object'},
               {'type': 'integer'}]}

On instance['name']:
    {'a': {'b': 12}}


1